# 👻 [프로젝트] 커스텀 프로젝트 직접 만들기
KLUE/BERT-base 모델 활용, NSMC Task

## STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성
## STEP 2. klue/bert-base model 및 tokenizer 불러오기
## STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기
## STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
## STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교


In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


## STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성

In [2]:
# Huggingface dataset에서 NSMC 데이터셋 불러오기
from datasets import load_dataset

huggingface_nsmc_dataset = load_dataset('nsmc')
print(huggingface_nsmc_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [3]:
# Train datasets의 각 컬럼에 해당하는 요소
train = huggingface_nsmc_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

### 비상비상 🚨🚨🚨

데이터가 너무 많아서 훈련할 때 10시간이나 기다렸어야 해서 데이터를 줄이는 방식을 택했다

In [4]:
from datasets import load_dataset

# 데이터셋 불러오기
huggingface_nsmc_dataset = load_dataset('nsmc')

# 데이터셋 크기를 줄이기 위해 10%만 샘플링
hf_train_dataset = huggingface_nsmc_dataset['train'].train_test_split(test_size=0.1, seed=42)['test']
hf_test_dataset = huggingface_nsmc_dataset['test'].train_test_split(test_size=0.1, seed=42)['test']

# 샘플링된 데이터셋 확인
print(hf_train_dataset)
print(hf_test_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached split indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-9d879241da83f708.arrow and /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-0fdc790e2ba4f643.arrow
Loading cached split indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5dcc18b03997bc21.arrow and /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-d0620966fea985b9.arrow


Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 15000
})
Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 5000
})


In [5]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




## STEP 2. klue/bert-base model 및 tokenizer 불러오기

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# klue/bert-base 모델과 토크나이저 불러오기
huggingface_tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

Huggingface에서는 AutoTokenizer와 AutoModel 기능을 제공하여, pretrained 모델의 경로 또는 이름만 알면 자동으로 모델을 생성할 수 있습니다.

예를 들어, BERT와 RoBERTa 모델을 사용할 때 각각의 토크나이저를 자동으로 선택할 수 있습니다. 특정 작업에 맞는 모델을 사용하기 위해 AutoModelForSequenceClassification을 권장하며, 다양한 모델을 실험할 수 있는 장점이 있습니다.

토크나이징은 transform 함수를 사용하여 데이터셋의 형태에 맞춰 진행하며, 문장이 길 경우 truncation을 통해 짧게 자를 수 있습니다.

In [7]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation=True,
        padding='max_length',
        return_token_type_ids=False,
        )

이렇게 하면 document 열을 기반으로 토큰화가 진행.

데이터셋을 한번에 토크나이징할때 자주 사용하는 기법은 map입니다.

map을 사용하게 되면 Data dictionary에 있는 모든 데이터들이 빠르게 적용시킬 수 있습니다.

우리는 map을 사용해 토크나이징을 진행하기 때문에 batch를 적용해야 되므로 batched=True로 주어야 합니다.

In [8]:

# 샘플링된 데이터셋에 transform 함수 적용
hf_train_dataset = hf_train_dataset.map(transform, batched=True)
hf_test_dataset = hf_test_dataset.map(transform, batched=True)

# 적용된 데이터셋 확인
print(hf_train_dataset)
print(hf_test_dataset)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
    num_rows: 15000
})
Dataset({
    features: ['attention_mask', 'document', 'id', 'input_ids', 'label'],
    num_rows: 5000
})


## STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [9]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

In [10]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_metric, load_dataset

# 전체 데이터셋에 토크나이징 적용
hf_train_dataset = hf_train_dataset.map(transform, batched=True)
hf_test_dataset = hf_test_dataset.map(transform, batched=True)

# 필요 없는 열 제거
hf_train_dataset = hf_train_dataset.remove_columns(['id', 'document'])
hf_test_dataset = hf_test_dataset.remove_columns(['id', 'document'])

# 분류를 위한 accuracy metric 설정
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Trainer 설정
trainer = Trainer(
    model=huggingface_model,                          # 학습할 모델
    args=training_arguments,              # 설정된 TrainingArguments
    train_dataset=hf_train_dataset,       # 학습 데이터셋
    eval_dataset=hf_test_dataset,         # 평가 데이터셋
    compute_metrics=compute_metrics       # 성능 평가 함수
)

# 학습 수행
trainer.train()

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5625


Epoch,Training Loss,Validation Loss,Accuracy
1,0.340800,0.311782,0.874800
2,0.226400,0.434459,0.877400
3,0.109700,0.596637,0.878800


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-2000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-2000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-2000/pytorch_mod

TrainOutput(global_step=5625, training_loss=0.2486114040798611, metrics={'train_runtime': 4903.6732, 'train_samples_per_second': 9.177, 'train_steps_per_second': 1.147, 'total_flos': 1.18399974912e+16, 'train_loss': 0.2486114040798611, 'epoch': 3.0})

In [11]:
trainer.evaluate(hf_test_dataset)

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8


{'eval_loss': 0.5966372489929199,
 'eval_accuracy': 0.8788,
 'eval_runtime': 170.9164,
 'eval_samples_per_second': 29.254,
 'eval_steps_per_second': 3.657,
 'epoch': 3.0}

In [24]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

## STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기
- 데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

In [25]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

# 저장 경로 설정
output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir=output_dir,                       # 모델이 저장될 경로
    evaluation_strategy="epoch",                 # 매 에포크마다 평가
    save_strategy="epoch",                       # 매 에포크마다 모델 저장
    learning_rate=2e-5,                          # 낮은 학습률을 사용해 세밀하게 조정
    per_device_train_batch_size=2,    A          # 학습 시 배치 크기를 16으로 설정
    per_device_eval_batch_size=2,               # 평가 시 배치 크기를 16으로 설정
    num_train_epochs=5,                          # 총 학습 에포크 수
    weight_decay=0.01,                           # 가중치 감소 설정
    logging_dir='./logs',                        # 로깅을 위한 디렉토리
    logging_steps=50,                            # 로그를 찍을 스텝 수
    save_total_limit=2,                          # 저장되는 체크포인트 개수 제한
    load_best_model_at_end=True,                 # 최고의 모델만을 저장
    gradient_accumulation_steps=8,   # 가상 배치 크기 증가
    fp16=True,   # 혼합 정밀도 사용
)

# Trainer 설정은 이전과 동일합니다.
trainer = Trainer(
    model=huggingface_model,                     # 학습할 모델
    args=training_arguments,                     # 설정한 TrainingArguments
    train_dataset=hf_train_dataset,           # 학습 데이터셋
    eval_dataset=hf_test_dataset,             # 평가 데이터셋
    compute_metrics=compute_metrics,             # 정확도 및 기타 평가 지표
)

# 모델 학습
trainer.train()

# 평가
eval_results = trainer.evaluate()
print("평가 결과:", eval_results)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp fp16 backend
***** Running training *****
  Num examples = 15000
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 4685


Epoch,Training Loss,Validation Loss,Accuracy
0,0.136200,0.527048,0.870200
1,0.122700,0.635770,0.876600
2,0.047600,0.817771,0.879800
3,0.034400,0.887424,0.879400
4,0.027200,0.925307,0.878400


/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1355: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  nn.utils.clip_grad_norm_(
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 2
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-937
Configuration saved in /aiffel/aiffel/transformers/checkpoint-937/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-937/pytorch_model.bin
Deleting older checkpoint [/aiffel/aiffel/transformers/checkpoint-500] due to args.save_total_limit
Deleting older checkpoint [/aiffel/aiffel/transformers/checkpoint-1000] due to args.save_total_limit
Deleting older checkpoint [/aiffel/aiffel/transformers/checkpoint-1500] due to args.save_to

평가 결과: {'eval_loss': 0.5270480513572693, 'eval_accuracy': 0.8702, 'eval_runtime': 190.5614, 'eval_samples_per_second': 26.238, 'eval_steps_per_second': 13.119, 'epoch': 5.0}


In [26]:
trainer.evaluate(hf_test_dataset)

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 2


{'eval_loss': 0.5270480513572693,
 'eval_accuracy': 0.8702,
 'eval_runtime': 190.6608,
 'eval_samples_per_second': 26.225,
 'eval_steps_per_second': 13.112,
 'epoch': 5.0}

fine-tuning을 했는데 accuracy가 조오금 더 떨어졌다!ㅎ.. 아무래도 데이터를 너무 줄어서 그런가보다

### 오류 !! 
런타임 오류: CUDA 메모리가 부족합니다. 20.00 MiB를 할당하려고 시도했습니다(GPU 0, 총 용량 14.58 GiB, 이미 할당된 13.28 GiB, 사용 가능한 1.56 MiB, PyTorch에서 총 13.37 GiB 예약).

가용할 수 있는 memory가 부족해서 생긴 오류인 것 같다 ㅜㅜ 남아있는 공간이 어느정도 인지 확인해보자.

In [18]:
!pip install psutil

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import psutil

def get_ram_info():
    ram_info = psutil.virtual_memory()
    total_ram = ram_info.total / (1024 ** 3)  # GB 단위
    available_ram = ram_info.available / (1024 ** 3)  # GB 단위
    used_ram = ram_info.used / (1024 ** 3)  # GB 단위
    ram_usage_percent = ram_info.percent
    
    print(f"Total RAM: {total_ram:.2f} GB")
    print(f"Available RAM: {available_ram:.2f} GB")
    print(f"Used RAM: {used_ram:.2f} GB")
    print(f"RAM Usage: {ram_usage_percent}%")

get_ram_info()

Total RAM: 17.57 GB
Available RAM: 12.19 GB
Used RAM: 4.98 GB
RAM Usage: 30.6%


In [22]:
import psutil

# 메모리 정보를 GiB 단위로 가져오기
total_memory = psutil.virtual_memory().total / (1024 ** 3)
available_memory = psutil.virtual_memory().available / (1024 ** 3)

print(f"Total Memory: {total_memory:.2f} GiB")
print(f"Available Memory: {available_memory:.2f} GiB")

Total Memory: 17.57 GiB
Available Memory: 12.17 GiB


메모리 확인 결과 사용가능한 메모리가 살짝 모자라서 torch.cuda.empty_cache()를 선언해서 해결했다.
또한, batch_size도 8에서 2로 낮춰서 최대로 낮게 돌려봤다..

## STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교
- STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교

1. Bucketing과 Dynamic Padding이란?
- Bucketing: 데이터를 유사한 길이별로 그룹화하여 배치를 구성하는 방식입니다. 모델이 패딩 토큰을 덜 처리하도록 하여 계산량을 줄입니다.
- Dynamic Padding: 배치마다 패딩을 동적으로 설정하여, 각 배치 내에서 가장 긴 문장의 길이만큼만 패딩을 적용해 불필요한 연산을 줄입니다.

2. group_by_length와 DataCollatorWithPadding 사용
- Trainer에서 bucketing과 dynamic padding을 구현하려면 group_by_length=True와 DataCollatorWithPadding를 설정하면 됩니다.

In [27]:
import os
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_metric

# 데이터 Collator - dynamic padding을 적용
data_collator = DataCollatorWithPadding(tokenizer=huggingface_tokenizer)

# 모델 평가를 위한 metric 설정
metric = load_metric("accuracy")  # 기존의 `compute_metrics` 함수로 정확도를 계산하는 metric입니다.

# Fine-tuning을 위한 TrainingArguments 설정 (Bucketing 및 Dynamic Padding 적용)
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # 필요 시 batch 크기 조정
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    group_by_length=True  # 길이별로 배치 구성 (Bucketing)
)

# Trainer 설정
trainer_with_bucketing = Trainer(
    model=huggingface_model,
    args=training_args,
    train_dataset=hf_train_dataset,
    eval_dataset=hf_test_dataset,
    tokenizer=huggingface_tokenizer,  # dynamic padding을 위한 tokenizer 전달
    data_collator=data_collator,      # dynamic padding을 적용하는 collator
    compute_metrics=compute_metrics
)

# 모델 학습
trainer_with_bucketing.train()

# 평가
eval_results_bucketing = trainer_with_bucketing.evaluate()
print("Bucketing 및 Dynamic Padding 적용 후 평가 결과:", eval_results_bucketing)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 15000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5625


Epoch,Training Loss,Validation Loss,Accuracy
1,0.032600,0.903095,0.868200
2,0.058800,0.870570,0.871200
3,0.022900,1.007563,0.874000


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1875
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1875/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1875/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/transformers/checkpoint-1875/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/transformers/checkpoint-1875/special_tokens_map.json
Deleting older checkpoint [/aiffel/aiffel/transformers/checkpoint-937] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-3750
Configuration saved in /aiffel/aiffel/transformers/checkpoint-3750/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-3750/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/transformers/checkpoint-3750/tokenizer_config.json

Bucketing 및 Dynamic Padding 적용 후 평가 결과: {'eval_loss': 0.8705695271492004, 'eval_accuracy': 0.8712, 'eval_runtime': 176.8874, 'eval_samples_per_second': 28.267, 'eval_steps_per_second': 3.533, 'epoch': 3.0}


### 첫 훈련, 파인튜닝, Bucketing 결과 정리

| |First Learning|Fine-tuning|Bucketing|
|---|---|---|---|
|시간|4903.6732s(81분)|4680s(78분)|4854s(80분)|
|acuuracy|0.8788|0.8702|0.8712|



동일한 조건은 원래 데이터의 10%만 남긴 train data 15000개, test data 5000개로 돌렸다.

첫 훈련과 파인튜닝, Bucketing 을 시도했을 때, 다이나믹하게 성능이 바뀌지는 않았다.

물론, 데이터의 개수가 너무 작게잡아서 그런 것 같지만. 시간이 없는 관계로 더 시도 해보지는 못하였다.

## 회고
🤗Hugging face로 모델을 받아와서 사용해보는 경험은 사실 경험이 많았다.

하지만, 쉽게 사용할 수 있다는 장점으로 허깅페이스에 대해서는 깊게 알고 있지는 않았는데

이번 고잉디퍼 학습에서 몰랐던 사실과 커스텀 데이터셋을 이용해 커스텀 모델을 가져와 프로젝트를 진행할 수 있다는 과정은 정말 좋았던 것 같다.

좋은 모델을 만들어 나도 내가 지은 커스텀 모델을 배포 해보고 싶다는 생각이 들었다:)